In [1]:

import math
from skimage import io
from skimage.transform import resize
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import numpy as np
import math

def imgToBinaryVector(image):
    image = resize(image, (20, 20), preserve_range=True).astype('uint8')
    threshold = 150
    # make all pixels < threshold black
    binarized = 1.0 * (image > threshold)

    # convert 3d array to 2d array
    return np.concatenate(binarized[:, :, 0])

def distance(v1, v2):
    return math.sqrt(
            sum(
                [ pow((v1-v2),2) for v1,v2 in zip(v1,v2) ]
            )
        )

def getTrainingDistanceForTestSample(X_train, test_sample):
    return [distance(train_sample,test_sample) for train_sample in X_train]

def get_most_frequent_element(l):
    return max(l, key=l.count)

def knn(X_train, Y_train, sample, k=3):
    training_disance = getTrainingDistanceForTestSample(X_train, sample)
    sorted_distance_indices = [ 
        pair[0]
        for pair in sorted(enumerate(training_disance), key=lambda x: x[1])
    ]

    candidates = [
        Y_train[idx]
        for idx in sorted_distance_indices[:k]
    ]
    
    top_candidate = get_most_frequent_element(candidates)
    return top_candidate


def getNumberOfFileName(s):
    return int(s.split("/", 1)[1][0])

def main():
    fileNames = []
    imageData = []
    for i in range(0,10):
        for x in range(1,11):
            fileNames.append(f'baseProjetOCR/{i}_{x}.png')
    print (fileNames)

    for file in fileNames:
            imageData = imageData.append({imgToBinaryVector(file):file})

    for key, value in ImageNameMap.items():
        ImageNameMaptest = ImageNameMap

        knn(imagesTest,)


ModuleNotFoundError: No module named 'skimage'